In [ ]:
# Setup
import sys
from pathlib import Path

notebook_dir = Path.cwd()
project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from core import (
    SocialSecurityModel,
    DemographicAssumptions,
    SocialSecurityReforms,
)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Initialize Social Security model
ss_model = SocialSecurityModel()

print("✅ Social Security Model initialized!")

---

## 1. How Social Security Works

### Funding Mechanism
- **Payroll Tax**: 12.4% on wages up to $168,600 (2024)
- Split: 6.2% employee + 6.2% employer
- Funds deposited into **OASI** (Old-Age & Survivors) and **DI** (Disability) trust funds

### Trust Fund Mechanics
- **Pay-as-you-go**: Current workers fund current retirees
- **Surplus years**: Excess invested in Treasury bonds
- **Trust fund balance**: ~$2.7 trillion (2024)
- **Interest income**: Earned on Treasury holdings

### Benefits
- Based on highest 35 years of earnings
- Progressive formula (replaces more income for low earners)
- COLA adjustments (Cost of Living)

In [ ]:
# Display current Social Security parameters
print("📊 Social Security Key Parameters (2024)")
print("=" * 50)
print(f"\n💰 Funding:")
print(f"   • Payroll tax rate: 12.4% (split equally)")
print(f"   • Taxable wage base: $168,600")
print(f"   • Annual revenue: ~$1.2 trillion")
print(f"\n👴 Benefits:")
print(f"   • Beneficiaries: ~70 million")
print(f"   • Average monthly benefit: ~$1,900")
print(f"   • Annual outlays: ~$1.4 trillion")
print(f"\n📈 Trust Fund:")
print(f"   • Current balance: ~$2.7 trillion")
print(f"   • Annual deficit: ~$200 billion")
print(f"   • Projected depletion: 2033-2035")

In [ ]:
# Run baseline trust fund projection
print("🔄 Running baseline projection (10 years, 1000 iterations)...")

baseline = ss_model.project_trust_fund(years=15, iterations=1000)

print("✅ Projection complete!")
print(f"\n📊 Results shape: {baseline['trust_fund_balance'].shape}")
print(f"   • Years: {baseline['trust_fund_balance'].shape[0]}")
print(f"   • Monte Carlo iterations: {baseline['trust_fund_balance'].shape[1]}")

In [ ]:
# Visualize trust fund trajectory
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

years = range(2024, 2024 + 15)

# Plot 1: Trust fund balance with uncertainty
ax1 = axes[0]
mean_balance = baseline['trust_fund_balance'].mean(axis=1) / 1e12
p5 = np.percentile(baseline['trust_fund_balance'], 5, axis=1) / 1e12
p95 = np.percentile(baseline['trust_fund_balance'], 95, axis=1) / 1e12

ax1.plot(years, mean_balance, 'b-', linewidth=2, label='Mean Projection')
ax1.fill_between(years, p5, p95, alpha=0.3, color='blue', label='90% CI')
ax1.axhline(y=0, color='red', linestyle='--', linewidth=2, label='Depletion')
ax1.set_xlabel('Year')
ax1.set_ylabel('Trust Fund Balance ($ Trillions)')
ax1.set_title('Social Security Trust Fund Projection', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Annual cash flow
ax2 = axes[1]
if 'revenue' in baseline and 'benefits' in baseline:
    revenue = baseline['revenue'].mean(axis=1) / 1e12
    benefits = baseline['benefits'].mean(axis=1) / 1e12
    ax2.plot(years, revenue, 'g-', linewidth=2, label='Revenue')
    ax2.plot(years, benefits, 'r-', linewidth=2, label='Benefits')
    ax2.fill_between(years, revenue, benefits, where=benefits>revenue, alpha=0.3, color='red', label='Deficit')
    ax2.set_xlabel('Year')
    ax2.set_ylabel('$ Trillions')
    ax2.set_title('Annual Revenue vs Benefits', fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
else:
    ax2.text(0.5, 0.5, 'Cash flow data not available', ha='center', va='center')

plt.tight_layout()
plt.show()

# Find depletion year
depletion_idx = np.where(mean_balance <= 0)[0]
if len(depletion_idx) > 0:
    print(f"\n⚠️ Projected Trust Fund Depletion: {2024 + depletion_idx[0]}")
else:
    print(f"\n✅ Trust fund remains solvent through projection period")

---

## 2. Why the Shortfall?

### Demographic Pressures

| Factor | Trend | Impact |
|--------|-------|--------|
| **Aging Population** | 65+ growing from 17% to 22% | More beneficiaries |
| **Life Expectancy** | +0.2 years/year | Longer benefit periods |
| **Birth Rate** | Declining | Fewer future workers |
| **Worker-to-Beneficiary Ratio** | 2.8→2.3 | Less revenue per beneficiary |

### Historical Context
- **1945**: 42 workers per beneficiary
- **1960**: 5 workers per beneficiary  
- **2024**: 2.8 workers per beneficiary
- **2040**: 2.3 workers per beneficiary (projected)

In [ ]:
# Visualize demographic pressures
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Worker-to-beneficiary ratio
ax1 = axes[0]
ratio_years = [1945, 1960, 1980, 2000, 2024, 2040]
ratios = [42, 5, 3.2, 3.4, 2.8, 2.3]
ax1.bar(range(len(ratio_years)), ratios, color=['green']*4 + ['orange'] + ['red'])
ax1.set_xticks(range(len(ratio_years)))
ax1.set_xticklabels(ratio_years)
ax1.set_ylabel('Workers per Beneficiary')
ax1.set_title('Worker-to-Beneficiary Ratio', fontweight='bold')
ax1.axhline(y=2.5, color='red', linestyle='--', alpha=0.5)

# Population 65+
ax2 = axes[1]
pop_years = [2000, 2010, 2020, 2030, 2040, 2050]
pop_65_plus = [12.4, 13.0, 16.5, 20.6, 21.6, 22.0]
ax2.plot(pop_years, pop_65_plus, 'r-', marker='o', linewidth=2)
ax2.set_xlabel('Year')
ax2.set_ylabel('% of Population')
ax2.set_title('Population 65+ (%)', fontweight='bold')
ax2.grid(True, alpha=0.3)

# Life expectancy at 65
ax3 = axes[2]
life_years = [1950, 1970, 1990, 2010, 2024, 2040]
life_exp = [13.9, 15.2, 17.2, 19.1, 20.0, 21.5]
ax3.plot(life_years, life_exp, 'b-', marker='s', linewidth=2)
ax3.set_xlabel('Year')
ax3.set_ylabel('Years')
ax3.set_title('Life Expectancy at 65', fontweight='bold')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 3. Reform Scenarios

There are several approaches to restoring Social Security solvency:

### Option 1: Benefit Adjustments
- Reduce benefits by 20-25%
- Raise retirement age
- Modify COLA formula

### Option 2: Revenue Increases
- Raise payroll tax rate (12.4% → 15.4%)
- Eliminate wage cap ($168,600)
- Apply to all income types

### Option 3: Mixed Approach
- Partial benefit reduction
- Partial revenue increase
- Phased implementation

In [ ]:
# Compare reform scenarios (if reform methods available)
print("📊 Reform Scenario Analysis")
print("=" * 50)

# Baseline projection
print("\n🔄 Running baseline (no reforms)...")
baseline_result = ss_model.project_trust_fund(years=20, iterations=500)
baseline_final = baseline_result['trust_fund_balance'].mean(axis=1)[-1] / 1e12
print(f"   Final balance (2044): ${baseline_final:.2f}T")

# Note: Reform projections would require specific model methods
print("\n💡 Reform options to explore:")
print("   1. Increase payroll tax to 15.4% (+3%)")
print("   2. Eliminate wage cap entirely")
print("   3. Reduce benefits by 20%")
print("   4. Raise full retirement age to 70")
print("   5. Mixed approach (smaller changes on both sides)")

---

## 4. Try It Yourself

Experiment with different projection parameters.

In [ ]:
# 🎮 EXPERIMENT: Custom projection
# ================================

# Adjust parameters
my_years = 25        # Try: 10, 20, 30
my_iterations = 500  # Try: 100, 500, 1000

# Run projection
my_result = ss_model.project_trust_fund(years=my_years, iterations=my_iterations)

# Analyze
my_mean = my_result['trust_fund_balance'].mean(axis=1) / 1e12
my_p5 = np.percentile(my_result['trust_fund_balance'], 5, axis=1) / 1e12
my_p95 = np.percentile(my_result['trust_fund_balance'], 95, axis=1) / 1e12

# Find depletion probability
final_balances = my_result['trust_fund_balance'][-1, :]
depletion_prob = (final_balances <= 0).mean() * 100

print(f"📊 {my_years}-Year Projection Results")
print(f"   • Final year mean balance: ${my_mean[-1]:.2f}T")
print(f"   • 90% CI: ${my_p5[-1]:.2f}T to ${my_p95[-1]:.2f}T")
print(f"   • Depletion probability: {depletion_prob:.1f}%")

---

## 🎓 What You've Learned

✅ Social Security is funded by 12.4% payroll tax on wages  
✅ The trust fund is projected to deplete around 2033-2035  
✅ Demographic pressures (aging, longer lives) are driving the shortfall  
✅ Reform options include benefit cuts, tax increases, or mixed approaches  
✅ PoliSim provides Monte Carlo projections with uncertainty ranges  

---

## 📚 Next Steps

| Notebook | Topic | Time |
|----------|-------|------|
| **05** | Monte Carlo & Uncertainty | 60-90 min |
| **06** | Tax Policy Modeling | 45-60 min |
| **07** | Policy Extraction | 30-45 min |

---

*Continue to Notebook 05: Monte Carlo & Uncertainty* →